In [238]:
import numpy as np
import sympy as sp
from scipy.linalg import eig

In [239]:
# Exercise Donadon 2S/2020 S1

#Properties (N and mm)
E1 = 129500
E2 = 9370
v12 = 0.38
G12 = 5240
t = 0.19
v21 = v12 * E2 / E1

# Plate Parameters
a = 300.
b = 200.
m = 3
n = 3

# Loads
Nxx = -1.
Nyy = 0
Nxy = 0

**[Q] in the principal direction**  pg. 52 chap 2 of the three regions

In [240]:
Q_0 = np.array([[E1/(1-v12*v21), v12*E2/(1-v12*v21), 0],
                 [v12*E2/(1-v12*v21), E2/(1-v12*v21),0],
                 [0, 0, G12]])

**[Q] rotated in theta direction** pg. 53 and 54 of chap 2

In [241]:
angles = [90, 90, 0, 90, 90, 0, 90, 90]
layup = zip(angles, ['f']*8)
Q_layup = []

#Layup assembly
for theta in layup:
    c = np.cos(theta[0]*np.pi/180)
    s = np.sin(theta[0]*np.pi/180)

    T_real = np.array([[c**2, s**2, 2*c*s],
                       [s**2, c**2, -2*c*s],
                       [-c*s, c*s, c**2-s**2]])

    T_engineering =  np.array([[c**2, s**2, c*s],
                               [s**2, c**2, -c*s],
                               [-2*c*s, 2*c*s, c**2-s**2]])


    if theta[1] == 'f':
        Q_layup.append((np.linalg.inv(T_real))@Q_0@T_engineering)


**[A], [B], [D]** pg. 24 fo chap 4 of the three regions

In [242]:
def t_position(t, layup):
    total_thickness = t * len(layup)

    current_z = -total_thickness/2
    ply_position = [current_z]
    for ply in layup:
        current_z += t
        ply_position.append(current_z)
    return ply_position

#t_position(t, angles)

In [243]:
t_position = t_position(t, angles)

A = np.zeros(9).reshape(3,3)
B = np.zeros(9).reshape(3,3)
D = np.zeros(9).reshape(3,3)

for i in enumerate(Q_layup):
    zk1 = t_position[i[0]+1]
    zk0 = t_position[i[0]]
    
    A += (zk1 - zk0)*i[1]
    B += (1/2)*(zk1**2 - zk0**2)*i[1]
    D += (1/3)*(zk1**3 - zk0**3)*i[1]


F = np.vstack([
    np.hstack([A, B]),
    np.hstack([B, D])
])

F = sp.Matrix(F)

### Rayleigh-Ritz formulation

In [244]:
x, y  = sp.symbols(['xi', 'eta'])

In [245]:
Sw = [[]]

for i in range(m):
    for j in range(n):
        #Sw[0].append(sp.sin((i+1)*sp.pi*x/a)*sp.sin((j+1)*sp.pi*y/b)) #BC == SS
        Sw[0].append(
            (1-sp.cos(2*(i+1)*sp.pi*x/a))*(1-sp.cos(2*(j+1)*sp.pi*y/b))) # BC == CC
        #Sw[0].append(x**(i+1)*y**(j+1)*(x-a)*(y-b))
Sw = sp.Matrix(Sw)


Sw_x = sp.diff(Sw, x)
Sw_y = sp.diff(Sw, y)

Sww = sp.Matrix([-1* sp.diff(Sw, x, x),
                -1 * sp.diff(Sw, y, y),
                -2 * sp.diff(Sw, x, y)])

In [246]:
size = m * n
KG = np.zeros(size**2).reshape(size, size)

if Nxx != 0:
    KGxx = sp.integrate(
         sp.integrate(
            Sw_x.T*Sw_x, (x, 0, a)), (y, 0, b)
    )
    KG += np.array(KGxx, dtype=np.longdouble)
if Nyy != 0:
    KGyy = sp.integrate(
         sp.integrate(
            Sw_y.T*Sw_y, (x, 0, a)), (y, 0, b)
    )
    KG += np.array(KGyy, dtype=np.longdouble)
if Nxy != 0:
    KGxy = sp.integrate(
         sp.integrate(
            Sw_x.T*Sw_y + Sw_x.T*Sw_x, (x, 0, a)), (y, 0, b)
    )
    KG += np.array(KGxy, dtype=np.longdouble)


In [247]:
KG.shape

(9, 9)

In [248]:
print(KG.shape)
print(KG.min())
print(KG.max())

(9, 9)
0.0
177.65287921960845


In [249]:
K = sp.integrate(
    sp.integrate(
        Sww.T * D * Sww, (x, 0, a)), (y, 0, b)
)

K = np.array(K, dtype=np.longdouble)

In [250]:
print(K.shape)
print(K.min())
print(K.max())

(9, 9)
0.0
131187.15733430977


In [251]:
A = K
B = KG

eig_values, eig_vectors = eig(A, B)

In [252]:
eig_values.min()

(21.979609263370477+0j)

In [253]:
eig_values = abs(eig_values)
eig_values.sort()
eig_values

array([   21.97960926,    31.3890807 ,    95.66761287,   146.90404866,
         319.26626763,   722.69186113,  1873.52529164,  2619.55038226,
       16378.66467807])

In [254]:
#eigvals, eigvecs = lb(A, B, silent=True)
#eigvals

In [255]:
np.unique(eig_values)[0]

21.979609263370477

In [256]:
eig_values[np.isnan(eig_values)] = 99999999999

In [257]:
eig_values = abs(eig_values)
eig_values[np.isnan(eig_values)] = 9999999999
min = eig_values.min()

index_min = np.where(eig_values == min)
eig_values[index_min]

array([21.97960926])

## Ploting

In [258]:
# This import registers the 3D projection, but is otherwise unused.
import matplotlib.pyplot as plt
from matplotlib import cm


def plot_eigenvalue(eig_values, eig_vectors, nth=0, ngridx=20, ngridy=20):
    eig_values = abs(eig_values)
    eig_values[np.isnan(eig_values)] = 99999999999

    if nth == 0:
        min_eig_value = eig_values.min()
    else:
        min_eig_value = np.unique(eig_values)[nth]
    
    print(f'{nth}th eigenvalue is {min_eig_value}')
    index_min = np.where(eig_values == min_eig_value)[0]
    C_values = eig_vectors[::, index_min]

    xi_arr = np.linspace(-1, 1, ngridx)
    eta_arr = np.linspace(-1, 1, ngridy)
    xi_mesh, eta_mesh = np.meshgrid(xi_arr, eta_arr)
    z = np.zeros(ngridx*ngridy).reshape(ngridx, ngridy)


    for i in range(ngridx):
        for j in range(ngridy):
            fw = Sw.subs(xi, xi_mesh[i,j]).subs(eta, eta_mesh[i,j])
            fw = np.array(fw).astype('float')
            wij = float((fw@C_values)[0,0])
            z[i,j] =  wij
    
    ax = plt.figure().add_subplot(projection='3d')
    surf = ax.plot_surface(xi_mesh, eta_mesh, z, cmap=cm.coolwarm)
    plt.show()


In [259]:
#%matplotlib notebook
#plot_eigenvalue(eig_values, eig_vectors, nth=0, ngridx=20, ngridy=20)

## Check with compmech

In [268]:
from compmech.panel.panel import Panel
from compmech.analysis import lb, static

In [269]:
# skin panels
laminaprop = (E1, E2, v12, G12, G12, G12)
stack = angles
p1 = Panel(group='skin', Nxx=-1, x0=0, y0=0., a=a, b=b,m=8, n=8, plyt=t, stack=stack, laminaprop=laminaprop)

In [270]:
# boundary conditions SSSS
p1.u1tx = 0 ; p1.u1rx = 1 ; p1.u2tx = 0 ; p1.u2rx = 1
p1.v1tx = 0 ; p1.v1rx = 1 ; p1.v2tx = 0 ; p1.v2rx = 1
p1.w1tx = 0 ; p1.w1rx = 1 ; p1.w2tx = 0 ; p1.w2rx = 1
p1.u1ty = 0 ; p1.u1ry = 1 ; p1.u2ty = 0 ; p1.u2ry = 1
p1.v1ty = 0 ; p1.v1ry = 1 ; p1.v2ty = 0 ; p1.v2ry = 1
p1.w1ty = 0 ; p1.w1ry = 1 ; p1.w2ty = 0 ; p1.w2ry = 1

# boundary conditions CCCC
p1.u1tx = 0 ; p1.u1rx = 0 ; p1.u2tx = 0 ; p1.u2rx = 0
p1.v1tx = 0 ; p1.v1rx = 0 ; p1.v2tx = 0 ; p1.v2rx = 0
p1.w1tx = 0 ; p1.w1rx = 1 ; p1.w2tx = 0 ; p1.w2rx = 1
p1.u1ty = 0 ; p1.u1ry = 0 ; p1.u2ty = 0 ; p1.u2ry = 0
p1.v1ty = 0 ; p1.v1ry = 0 ; p1.v2ty = 0 ; p1.v2ry = 0
p1.w1ty = 0 ; p1.w1ry = 1 ; p1.w2ty = 0 ; p1.w2ry = 1

In [271]:
k0_1 = p1.calc_k0()
kG0_1 = p1.calc_kG0()

eigvals, eigvecs = lb(k0_1, kG0_1, silent=True)
eigvals

		Calculating k0... 
		finished!
		Calculating kG0... 
		finished!


array([   9.72942675,   10.88748912,   15.62106123,   21.86783299,
         39.20868828,   48.46531024,   57.23148202,   71.63016224,
         87.47368308,   98.86985417,  110.11775697,  117.73354439,
        125.27854215,  146.901481  ,  202.66831226,  219.97214369,
        236.98604408,  314.54865674,  374.92806848,  421.48157519,
        628.14138622, 1044.70674471, 1364.15756868, 1552.4440289 ,
       1619.71040419])

In [264]:
print(k0_1.min())
print(k0_1.max())

-18.63330883479195
113.43444399800934


In [265]:
print(kG0_1.min())
print(kG0_1.max())

-0.0008465608465608449
0.0006349206349206358


In [266]:
#sp.Matrix(kG0_1.toarray())

In [267]:
#sp.Matrix(KG)